 # Tema 6: Decision Tree
 ----
  
University: UTEC \\
   Course: Machine Learning \\
   Professor: Cristian López Del Alamo \\
   Topic: Decision Trees \\
  

  ----

  Names and Surnames of Members: (Do not forget to enter the % participation)
  - Member 1:
  - Member 2:
  - Member 3:
  - Member 4:
  



For this exercise, you will need to build your own decision tree.

- Please note that this is a recursive algorithm.
- The base case occurs when all the elements of a node have the same labels, that is, it is a terminal node. Then the label of that node takes the value of the common label.
- In the case that it is not a terminal node, the algorithm must search for one of the
feactures by which to divide and for this use Information Gain (Entropy or Gini).
- Split the dataset using the feature that generates a greater information gain in the parent or a lower GINI and recursively call the create_DT function.

You will use the iris database, with 4 features and 3 classes.
Randomly take 80% of the data to create the tree and the rest to
test the accuracy of the prediction.
Finally, it shows the **accuracy** of its model using a confusion matrix.

----

Descarge el [Dataset](https://gist.github.com/Thanatoz-1/9e7fdfb8189f0cdf5d73a494e4a6392a)

----


Trabaje en equipo:

[Link de apoyo 1](https://towardsdatascience.com/the-simple-math-behind-3-decision-tree-splitting-criterions-85d4de2a75fe)

[Link de apoyo 2](https://www.quantstart.com/articles/Beginners-Guide-to-Decision-Trees-for-Supervised-Machine-Learning/)

----
## In the following code, your group can add functions, remove functions, or modify the function parameters.
---

In [11]:
import seaborn as sns
import numpy as np
import pandas as pd

iris = sns.load_dataset('iris')
print(iris.head())


class Nodo:
 #Define what your data members will be
  def __init__(self, X, Y, index):
    self.m_X = X
    self.m_Y = Y
    self.m_Index = index
    self.m_HijoIzquierdo = None
    self.m_HijoDerecho = None
    self.m_Feature = None
    self.m_Valor = None

  def IsTerminal(self,Y):
    # return true if this node has the same labels in Y
    return len(np.unique(Y)) == 1


  def BestSplit(self,X,Y):
    H = self.Entropy(Y)
    for i in range(X.shape[1]):
      feature_values = np.unique(X[:, i])
      for value in feature_values:
        X_izquierdo = X[X[:, i] <= value]
        Y_izquierdo = Y[X[:, i] <= value]
        H_izquierdo = self.Entropy(Y_izquierdo)

        X_derecho = X[X[:, i] > value]
        Y_derecho = Y[X[:, i] > value]
        H_derecho = self.Entropy(Y_derecho)
        H_condicional = (len(Y_izquierdo) / len(Y)) * H_izquierdo + (len(Y_derecho) / len(Y)) * H_derecho
        if H_condicional < H:
          self.m_Feature = i
          self.m_Valor = value
          H = H_condicional

    return self.m_Feature, self.m_Valor

  def Entropy(self,Y):
    p = np.unique(Y, return_counts=True)[1] / len(Y)
    H = 0
    for pi in p:
      if pi != 0:
        H -= pi * np.log2(pi)
    return H

  def Gini(self,Y):
    p = np.unique(Y, return_counts=True)[1] / len(Y)
    return 1 - np.sum(p ** 2)


class DT:
 # Defina cuales será sus mimbros datos
    def __init__(self, X, Y, index):
        self.m_X = X
        self.m_Y = Y
        self.m_Index = index
        self.m_Root = None

    def create_DT(self):
        self.m_Root = Nodo(self.m_X, self.m_Y, self.m_Index)
        self.create_DT_recursivo(self.m_Root)

    def create_DT_recursivo(self, nodo: Nodo):
        if nodo.IsTerminal(nodo.m_Y):
            return
        nodo.m_Feature, nodo.m_Valor = nodo.BestSplit(nodo.m_X, nodo.m_Y)
        nodo.m_HijoIzquierdo = Nodo(nodo.m_X[nodo.m_X[:, nodo.m_Feature] <= nodo.m_Valor],
                                    nodo.m_Y[nodo.m_X[:, nodo.m_Feature] <= nodo.m_Valor],
                                    nodo.m_Index)
        nodo.m_HijoDerecho = Nodo(nodo.m_X[nodo.m_X[:, nodo.m_Feature] > nodo.m_Valor],
                                  nodo.m_Y[nodo.m_X[:, nodo.m_Feature] > nodo.m_Valor],
                                  nodo.m_Index)
        self.create_DT_recursivo(nodo.m_HijoIzquierdo)
        self.create_DT_recursivo(nodo.m_HijoDerecho)

    def Find_Best_Split(self):
        self.m_Root.BestSplit(self.m_X, self.m_Y)

    def predict(self, X):
        return self.predict_recursivo(X, self.m_Root)
    
    def predict_recursivo(self, X, nodo: Nodo):
        if nodo.IsTerminal(nodo.m_Y):
            return np.argmax(np.bincount(nodo.m_Y))
        if X[nodo.m_Feature] <= nodo.m_Valor:
            return self.predict_recursivo(X, nodo.m_HijoIzquierdo)
        else:
            return self.predict_recursivo(X, nodo.m_HijoDerecho)

   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa


In [12]:
# Test with iris_dataset.csv that is in the same folder
iris = pd.read_csv('iris_dataset.csv')
iris = iris.sample(frac=1).reset_index(drop=True)
X = iris.iloc[:, :-1].values
Y = iris.iloc[:, -1].values
index = np.arange(len(Y))
np.random.shuffle(index)
X_train = X[index[:int(0.7 * len(Y))]]
Y_train = Y[index[:int(0.7 * len(Y))]]
X_test = X[index[int(0.7 * len(Y)):]]
Y_test = Y[index[int(0.7 * len(Y)):]]
dt = DT(X_train, Y_train, np.arange(len(Y_train)))
dt.create_DT()
print(dt.predict(X_test[0]))

TypeError: Cannot cast array data from dtype('O') to dtype('int64') according to the rule 'safe'

Colocar datos de su equipo y su colab aquí: [link ](https://docs.google.com/spreadsheets/d/1RhFmHVXiWbHd4opNn1IX158vTqFFZlADMma0WJ0QA3s/edit?usp=sharing)

[Lectura Extra ](https://www.cs.toronto.edu/~axgao/cs486686_f21/lecture_notes/Lecture_07_on_Decision_Trees.pdf)